In [2]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions

In [3]:
from azure.cli.core import get_default_cli

def az_cli (args_str):
    args = args_str.split()
    cli = get_default_cli()
    cli.invoke(args)
    if cli.result.result:
        return cli.result.result
    elif cli.result.error:
        raise cli.result.error
    return True

In [4]:
RES_GROUP='seobot-resources-dev'
ACCT_NAME='seobot-dev-cosmos'

args = "cosmosdb show --resource-group {} --name {} --query documentEndpoint --output tsv".format(RES_GROUP, ACCT_NAME)
ACCOUNT_URI = az_cli(args)
args = "cosmosdb keys list --resource-group {} --name {} --query \"primaryMasterKey\" --output tsv".format(RES_GROUP, ACCT_NAME)
ACCOUNT_KEY = az_cli(args)

https://seobot-dev-cosmos.documents.azure.com:443/
NROrQMOzdTLXvUNMUFRGvys50Wxsj948HAn1HRyy4H6EF8GOGazfKZE3Nef7y0NRVcj0fNF9vvnC3Stw9I8iRQ==


In [5]:
client = CosmosClient(ACCOUNT_URI, {'masterKey': ACCOUNT_KEY})

In [6]:
database_name = 'seo-stats'
container_name = 'search_stats'
database = client.get_database_client(database_name)
print("Got database {}".format(database_name))
container = database.get_container_client(container_name)
print("Got container {}".format(container_name))

Got database seo-stats
Got container search_stats


In [7]:
## Run a query against the container to see number of documents
query = 'SELECT VALUE COUNT(1) FROM c'
result = list(container.query_items(query, enable_cross_partition_query=True))

print('Container with id \'{0}\' contains \'{1}\' items'.format(container.id, result[0]))

Container with id 'search_stats' contains '754' items


In [8]:
query = 'SELECT c.searchDate, c.queryString, c.resultPage FROM c WHERE c.resultPage > 0'
result = list(container.query_items(query, enable_cross_partition_query=True))

In [9]:
import pandas as pd
df_cosmos = pd.DataFrame(result)
# Sample the top 10 results
df_cosmos.head(10)

,searchDate,queryString,resultPage
0,2021-04-08T00:01:39.2661834Z,palm beach acupuncture,1
1,2021-04-08T00:02:04.1421945Z,Shoni Shin palm beach,1
2,2021-04-08T00:03:09.5210883Z,Moxibustion palm beach,1
3,2021-04-08T00:17:04.8936776Z,Shoni Shin palm beach,1
4,2021-04-08T00:18:41.0905885Z,Shoni Shin palm beach,1
5,2021-04-08T00:19:09.4503053Z,Moxibustion palm beach,1
6,2021-04-08T00:26:04.2451235Z,palm beach acupuncture,1
7,2021-04-08T00:40:13.4984449Z,Gua Sha palm beach,1
8,2021-04-08T01:42:39.2499031Z,Shoni Shin palm beach,1
9,2021-04-08T02:32:24.9932343Z,HYPNOBIRTHING palm beach,1


## Mean Page Results Over All Time

In [10]:
df_rank = df_cosmos.groupby("queryString").mean().reset_index()
display(df_rank.head(20))

,queryString,resultPage
0,Gua Sha palm beach,1
1,HYPNOBIRTHING palm beach,1
2,Moxibustion palm beach,1
3,Shoni Shin palm beach,1
4,chaas gantt acupunture,1
5,palm beach acupuncture,1


In [11]:
## What are the top 5 query strings?
pd.DataFrame(df_cosmos[df_cosmos['resultPage']>=0].groupby('queryString').size().sort_values(ascending=False).head(5), columns=['Count'])

,Count
queryString,
Shoni Shin palm beach,49
HYPNOBIRTHING palm beach,46
chaas gantt acupunture,37
palm beach acupuncture,36
Moxibustion palm beach,34


In [12]:
import sys
!{sys.executable} -m pip install bokeh --user

In [13]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral3
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, FactorRange

# Get the top 20 items as an array
top_20_items = df_cosmos[df_cosmos['resultPage']>=0].groupby('queryString').size().sort_values(ascending=False)[:20].index.values.tolist()
#print(top_20_items)

# Filter our data to only these 20 items
df_top20 = df_cosmos[df_cosmos['queryString'].isin(top_20_items)]
#df_top20.head(20)

# Group by queryString and searchDate, sorting by resultPage
#df_top10_sorted = df_top10.groupby(['Item', 'Action']).count().rename(columns={'Country':'ResultCount'}, inplace=False).reset_index().sort_values(['Item', 'ResultCount'], ascending = False).set_index(['Item', 'Action'])
df_top20_sorted = df_top20.groupby(['queryString']).mean().reset_index().sort_values(['resultPage'], ascending = True).set_index(['queryString'])
#df_top20_sorted.head(20)

# Get sorted X-axis values
x_axis_values = df_top20_sorted.index.values.tolist()
#print(x_axis_values)

group = df_top20_sorted.groupby(['queryString'])
#group.head(20)

# Specifiy colors for X axis
index_cmap = factor_cmap('queryString', palette=Spectral3, factors=sorted(df_top20.queryString.unique()), start=1, end=2)

# Create the plot

p = figure(plot_width=1200, plot_height=500, title="Page Results from Search Queries", x_range=FactorRange(*x_axis_values), toolbar_location=None, tooltips=[("Average Page Result", "@resultPage"), ("queryString", "@queryString")])

p.vbar(x='queryString', top='resultPage_max', width=1, source=group,
        line_color="white", fill_color=index_cmap, )

#Configure how the plot looks
p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = "black"
p.xaxis.axis_label = "Query"
p.yaxis.axis_label = "Page"

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)

Loading BokehJS ...